<a href="https://colab.research.google.com/github/MadmanMarble/Planet_U-NET/blob/main/R_matrix_analysis_plotting_20240326.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install.packages(c('raster','sf','fasterize'))
install.packages("googledrive")
install.packages("devtools")

devtools:::install_github("gearslaboratory/gdalUtils")

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘e1071’, ‘sp’, ‘Rcpp’, ‘terra’, ‘classInt’, ‘s2’, ‘units’, ‘wk’, ‘RcppArmadillo’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘promises’, ‘shiny’, ‘downlit’, ‘htmlwidgets’, ‘miniUI’, ‘pkgdown’, ‘profvis’, ‘urlchecker’





R.methodsS3 (NA -> 1.8.2 ) [CRAN]
R.oo        (NA -> 1.26.0) [CRAN]
iterators   (NA -> 1.0.14) [CRAN]
R.utils     (NA -> 2.12.3) [CRAN]
foreach     (NA -> 1.5.2 ) [CRAN]


Skipping 1 packages not available: rgdal

Installing 6 packages: R.methodsS3, R.oo, iterators, rgdal, R.utils, foreach

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘rgdal’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpJ3sSiA/remotes10c6a2c1125/gearslaboratory-gdalUtils-8872aaf/DESCRIPTION’ ... OK
* preparing ‘gdalUtils’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘gdalUtils_2.0.3.2.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘/tmp/RtmpJ3sSiA/file10c787e6da2/gdalUtils_2.0.3.2.tar.gz’ had non-zero exit status”


In [2]:
library(googledrive)

In [9]:
library(raster)

Loading required package: sp



In [10]:
library(magrittr)


Attaching package: ‘magrittr’


The following object is masked from ‘package:raster’:

    extract




In [3]:
drive_auth(use_oob = TRUE, cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=ab9ccb710e3b1c94c72874d8cbfbdc99&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQWVhWVNIQjQzVlBWeWo0bjZlVWhBZFNlNmFUM2tIYmM3WkpWTUxfSkRKZ0dfd0lMLU0xaU1VMW5qQ3VJelZWR200STFyZyIsInN0YXRlIjoiYWI5Y2NiNzEwZTNiMWM5NGM3Mjg3NGQ4Y2JmYmRjOTkifQ==


In [4]:
# https://drive.google.com/file/d/1tKj2X7NQELkArrQ8K8ptsk1YUi5QXs3c/view?usp=drive_link
drive_download(file = as_id('1tKj2X7NQELkArrQ8K8ptsk1YUi5QXs3c'), path =  '/content/oneYear.zip', overwrite = TRUE)
# one <- st_read('/content/oneYear.zip')

File downloaded:

• model_03222023_1255-20240326T202803Z-001.zip
  <id: 1tKj2X7NQELkArrQ8K8ptsk1YUi5QXs3c>

Saved locally as:

• /content/oneYear.zip



In [5]:
unzip("/content/oneYear.zip", files = NULL, list = FALSE, overwrite = TRUE,
      junkpaths = FALSE, exdir = ".", unzip = "internal",
      setTimes = FALSE)

In [8]:
file_list <-list.files("/content/model_03222023_1255", pattern = "*.tif", full.names = TRUE)

In [32]:
file_list[[50]]

[1] "/content/model_03222023_1255/epoch_41_1530_planet_stack_utm4.tif"

In [34]:
raster_list <- file_list[50] %>% lapply(stack)

In [35]:
raster_list

[[1]]
class      : RasterStack 
dimensions : 384, 384, 147456, 1  (nrow, ncol, ncell, nlayers)
resolution : 1, 1  (x, y)
extent     : 628182, 628566, 2365605, 2365989  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=4 +datum=NAD83 +units=m +no_defs 
names      : epoch_41_1530_planet_stack_utm4 



In [36]:
unzip("/content/test_labels-20240326T203837Z-001.zip", files = NULL, list = FALSE, overwrite = TRUE,
      junkpaths = FALSE, exdir = ".", unzip = "internal",
      setTimes = FALSE)

In [37]:
labels_file_list <-list.files("/content/test_labels", pattern = "*.tif", full.names = TRUE)

In [38]:
labels_file_list

[1] "/content/test_labels/1530_nwi_ccap_filled_utm4.tif"
[2] "/content/test_labels/1532_nwi_ccap_filled_utm4.tif"
[3] "/content/test_labels/1546_nwi_ccap_filled_utm4.tif"
[4] "/content/test_labels/1557_nwi_ccap_filled_utm4.tif"
[5] "/content/test_labels/2118_nwi_ccap_filled_utm4.tif"
[6] "/content/test_labels/2355_nwi_ccap_filled_utm4.tif"
[7] "/content/test_labels/823_nwi_ccap_filled_utm4.tif"

In [39]:
labels_raster_list <- labels_file_list[1] %>% lapply(stack)

In [40]:
library(data.table)

In [41]:
val_dt <- data.table(truth = values(labels_raster_list[[1]]),prediction = values(raster_list[[1]]))

In [42]:
# install.packages("caret")

In [43]:
# library(caret)

In [46]:
confus_dt <- confusionMatrix(val_dt$truth %>% factor(levels = 0:12), val_dt$prediction %>% factor(levels = 0:12))

In [51]:
byClass_dt <- confus_dt$byClass %>% data.table

In [52]:
byClass_dt$Sensitivity

[1]       NA 0.000000       NA       NA       NA       NA       NA 0.000000
 [9]       NA       NA 0.492502       NA       NA

In [53]:
library(ggplot2)

In [ ]:
prettyConfused<-function(Actual,Predict,colors=c("white","red4","dodgerblue3"),text.scl=5){
  actual = as.data.frame(table(Actual))
  names(actual) = c("Actual","ActualFreq")

  #build confusion matrix
  confusion = as.data.frame(table(Actual, Predict))
  names(confusion) = c("Actual","Predicted","Freq")

  #calculate percentage of test cases based on actual frequency

  confusion = merge(confusion, actual, by=c('Actual','Actual'))
  confusion$Percent = confusion$Freq/confusion$ActualFreq*100
  confusion$ColorScale<-confusion$Percent*-1
  confusion[which(confusion$Actual==confusion$Predicted),]$ColorScale<-confusion[which(confusion$Actual==confusion$Predicted),]$ColorScale*-1
  confusion$Label<-paste(round(confusion$Percent,0),"%, n=",confusion$Freq,sep="")
  tile <- ggplot() +
    geom_tile(aes(x=Actual, y=Predicted,fill=ColorScale),data=confusion, color="black",size=0.1) +
    labs(x="Actual",y="Predicted")

  tile = tile +
        geom_text(aes(x=Actual,y=Predicted, label=Label),data=confusion, size=text.scl, colour="black") +
        scale_fill_gradient2(low=colors[2],high=colors[3],mid=colors[1],midpoint = 0,guide='none')
}